In [6]:
# Step 2: Set up mlflow tracking server
import mlflow

mlflow.set_tracking_uri("http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/")

In [7]:
# Set or create an experiment
mlflow.set_experiment("Exp 5: ML Algos with HP Tuning")

<Experiment: artifact_location='s3://yt-plugin-bucket/201394593778390276', creation_time=1744190402646, experiment_id='201394593778390276', last_update_time=1744190402646, lifecycle_stage='active', name='Exp 5: ML Algos with HP Tuning', tags={}>

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [9]:
df = pd.read_csv('../data/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [10]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1','l2'])    

    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for LightGBM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = model = LogisticRegression(C=best_params['C'], penalty=best_params["penalty"], solver='liblinear', random_state=42)
    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for LightGBM
run_optuna_experiment()


[I 2025-04-09 15:59:23,259] A new study created in memory with name: no-name-c7c74676-92b0-47b8-938f-6a265d9166a0
[I 2025-04-09 15:59:23,345] Trial 0 finished with value: 0.6388712745719721 and parameters: {'C': 0.0002278318227861861, 'penalty': 'l2'}. Best is trial 0 with value: 0.6388712745719721.
[I 2025-04-09 15:59:23,496] Trial 1 finished with value: 0.7452969773832171 and parameters: {'C': 0.15441689030075012, 'penalty': 'l2'}. Best is trial 1 with value: 0.7452969773832171.
[I 2025-04-09 15:59:23,660] Trial 2 finished with value: 0.7480448108222363 and parameters: {'C': 0.17616619103433717, 'penalty': 'l2'}. Best is trial 2 with value: 0.7480448108222363.
[I 2025-04-09 15:59:23,710] Trial 3 finished with value: 0.3333333333333333 and parameters: {'C': 0.0007114654852246988, 'penalty': 'l1'}. Best is trial 2 with value: 0.7480448108222363.
[I 2025-04-09 15:59:23,914] Trial 4 finished with value: 0.7838723314309871 and parameters: {'C': 1.9259015355391158, 'penalty': 'l1'}. Best i

🏃 View run LogisticRegression_SMOTE_TFIDF_Trigrams at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/201394593778390276/runs/923b8e121f1f4270910527d5583e9bd0
🧪 View experiment at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/201394593778390276
